# Auto Agent Builder

This AI Agent provides you with the code for Automating you are day to day tasks

In [ ]:
import asyncio

from google.adk.agent import Llmagent, SequentialAgent
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types


In [ ]:
try:
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GOOGLE_API_KEY)
    gemini_model = Gemini(model= "gemini-2.5-flash")
    print("✓ API Key Configured")
except Exception as e:
    print(f"⚠ API Key Error: {str(e)}")
    print("📌 To fix: Go to Add-ons → Secrets → Add 'GOOGLE_API_KEY'")
    GOOGLE_API_KEY = None

**Agents**

In [ ]:
task_breakdown_agent = LlmAgent(
    name= "TaskBreakdownAgent",
    model= gemini_model,
    instruction= """
You are a Task Breakdown Agent in an AI Automation Builder system.

Your job is to:
1. Read the user's raw automation request.
2. Decide whether the task is:
   - Technically automatable
   - Safe and legal
   - Feasible using scripting or APIs
3. If the task is NOT automatable:
   - Clearly explain why.
   - Do NOT proceed further.
4. If the task IS automatable:
   - Break the task into 3 to 6 logical execution steps.
   - Each step must be:
     - Clear
     - Actionable
     - Free from implementation details
5. Do NOT suggest programming languages.
6. Do NOT suggest tools or libraries.
7. Do NOT generate any code.
8. Output only a numbered step list.

Your output must look like:
1. Step one description
2. Step two description
3. Step three description

""",
output_key= "task_steps"
)

task_planner_agent = LlmAgent(
    name= "TaskPlannerAgent",
    model= gemini_model,
    instruction= """
You are a Task Planner Agent in an AI Automation Builder system.

Your job is to:
1. Take the execution steps generated by the Task Breakdown Agent.
2. Decide:
   - The best programming language to use.
   - The automation approach (script, API, browser automation, file system, etc.).
3. Select:
   - Required libraries
   - Required tools
   - Any external services or APIs needed
4. Convert the high-level steps into a detailed technical execution plan.
5. Explain:
   - Why the chosen language is suitable.
   - Why the selected tools are required.
6. Do NOT generate actual code.
7. Do NOT simplify the task.
8. Do NOT change the original task intent.

Your output must strictly contain:
- Programming Language:
- Execution Strategy:
- Required Libraries:
- External Tools/APIs (if any):
- Step-by-step Technical Plan:

""",
    output_key= "planner_steps"
)

code_gen_agent = LlmAgent(
    name= "CodeGeneratorAgent",
    model= gemini_model,
    instruction= """
You are a Code Generator Agent in an AI Automation Builder system.

Your job is to:
1. Read the technical execution plan provided by the Task Planner Agent.
2. Generate complete, executable, production-ready automation code.
3. Follow these strict rules:
   - Use only the selected programming language.
   - Use only the selected libraries.
   - Do NOT invent new tools.
4. The code must include:
   - Proper imports
   - Error handling
   - Clear function structure
   - Comments for understanding
5. The code must be:
   - Copy-paste runnable
   - Free from placeholders like "your_api_key_here"
   - Free from pseudo code
6. After the code, generate:
   - Setup instructions (pip install etc.)
   - How to run the script
7. Do NOT explain planning logic.
8. Do NOT modify the original task.

Your output format must be:

================ CODE ================
<full runnable code>

================ SETUP ================
<installation instructions>

================ RUN ================
<run instructions>

""",
    code_executor= BuiltInCodeExecutor,
    output_key= "code_gen"
)

Root **Agent**

In [ ]:
sequentialAgent = SequentialAgent(
    name= "AutomationAgent",
    instruction= prompt.ROOT_AGENT,
    sub_agents= [task_breakdown_agent, task_planner_agent, code_gen_agent]
)

root_agent = sequentialAgent
session_service = InMemorySessionService()

runner = Runner(
    agent= root_agent,
    app_name= "AutomationCodeGenerator",
    session_service= session_service,
)

**Run Function**

In [ ]:
async def run_agent(user_input: str):
    TASK_ID = "task-001"
    SESSION_ID = "session-1"
    APP_NAME = "AutomationCodeGenerator"

    content = types.Content(
        role= "user",
        parts= [types.Part(text=user_input)]
    )

    events = runner.run(
        user_id= "user-1",
        session_id= "session-1",
        new_message= content
    )

    final_output = None

    for event in events:
        if event.is_final_response():
            final_output = event.content.parts[0].text

    session = session_service.get_session(
        app_name= APP_NAME,
        user_id= "user-1",
        session_id= SESSION_ID,
    )

    current_state = dict(session.state)

    if "task_steps" in current_state:
        send_a2a_message(
            sender="TaskBreakdownAgent",
            receiver="TaskPlannerAgent",
            task_id=TASK_ID,
            payload={"task_steps": current_state["task_steps"]}
        )

    if "planner_steps" in current_state:
        send_a2a_message(
            sender="TaskPlannerAgent",
            receiver="CodeGeneratorAgent",
            task_id=TASK_ID,
            payload={'planner_steps': current_state["planner_steps"]}
        )


    print(final_output)

    print("Session State")
    for k, v in session.state.items():
        print(f"{k}: \n {v}\n")

    print("\n A2A Message Trace")
    for msg in A2A_BUS:
        print(msg.model_dump())

if __name__ == '__main__':
    user_task = "Create a Python script to rename all files in a folder to timestamps"
    asyncio.run(run_agent(user_task))